In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [2]:
training_data = [
    
     ['Red','Sports','Domestic','Yes'],
     ['Red','Sports','Domestic','No'],
     ['Red','Sports','Domestic','Yes'],
     
     ['Yellow','Sports','Domestic','No'],
     ['Yellow','Sports','Imported','Yes'],
    
     ['Yellow','SUV','Imported','No'],
     ['Yellow','SUV','Imported','Yes'],
     ['Yellow','SUV','Domestic','No'],
    
     ['Red','SUV','Imported','No'],
     ['Red','Sports','Imported','Yes'],
    
]

In [3]:

header = ["Color", "Type", "Origin","Stolen"]

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [5]:
unique_vals(training_data, 0)

{'Red', 'Yellow'}

In [6]:
unique_vals(training_data, 1)

{'SUV', 'Sports'}

In [7]:
unique_vals(training_data, 2)

{'Domestic', 'Imported'}

In [8]:
unique_vals(training_data, 3)

{'No', 'Yes'}

In [9]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [10]:
#######
# Demo:
class_counts(training_data)
#######

{'Yes': 5, 'No': 5}

In [11]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [12]:
is_numeric(7)

True

In [13]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [16]:
q=Question(1, 'Sports')
q

Is Type == Sports?

In [17]:
example = training_data[0]
q.match(example)

In [17]:
qq1=Question(0, 'Red')
qq1


True

In [19]:
r1=Question(2,'Sports')
r1

Is Origin == Sports?

In [20]:
example1 = training_data[0]
r1.match(example1)

False

In [22]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [23]:
true_rows, false_rows = partition(training_data, Question(1, 'Sports'))
true_rows

[['Red', 'Sports', 'Domestic', 'Yes'],
 ['Red', 'Sports', 'Domestic', 'No'],
 ['Red', 'Sports', 'Domestic', 'Yes'],
 ['Yellow', 'Sports', 'Domestic', 'No'],
 ['Yellow', 'Sports', 'Imported', 'Yes'],
 ['Red', 'Sports', 'Imported', 'Yes']]

In [24]:
false_rows

[['Yellow', 'SUV', 'Imported', 'No'],
 ['Yellow', 'SUV', 'Imported', 'Yes'],
 ['Yellow', 'SUV', 'Domestic', 'No'],
 ['Red', 'SUV', 'Imported', 'No']]

In [25]:
def gini(rows):

    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [26]:
no_mixing = [['Yes'],
              ['Yes']]
gini(no_mixing)

0.0

In [27]:
no_mixing = [['Yes'],
              ['No']]
gini(no_mixing)

0.5

In [28]:
def info_gain(left, right, current_uncertainty):
   
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [29]:

current_uncertainty = gini(training_data)
current_uncertainty

0.5

In [32]:
# How much information do we gain by partioning on 'Red'?
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.020000000000000018

In [33]:
# What about if we partioned on 'Yellow' instead?
true_rows, false_rows = partition(training_data, Question(0,'Yellow'))
info_gain(true_rows, false_rows, current_uncertainty)

0.020000000000000018

In [34]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [35]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is Type == Sports?

In [37]:
class Leaf:
  

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [38]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [39]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [40]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [41]:
my_tree = build_tree(training_data)

In [42]:
print_tree(my_tree)

Is Type == Sports?
--> True:
  Is Origin == Domestic?
  --> True:
    Is Color == Red?
    --> True:
      Predict {'Yes': 2, 'No': 1}
    --> False:
      Predict {'No': 1}
  --> False:
    Predict {'Yes': 2}
--> False:
  Is Origin == Domestic?
  --> True:
    Predict {'No': 1}
  --> False:
    Is Color == Yellow?
    --> True:
      Predict {'No': 1, 'Yes': 1}
    --> False:
      Predict {'No': 1}
